In [2]:
import pandas as pd
import numpy as np
import ast
import re
import chardet
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

%run "./support_functions.ipynb"

file_path = "./Starting_Datas/BPMN.csv"
output_file_path = "./Output_Files/BPMN_cleaned.csv"

removed_rows_file_path = "./Discarded_Datas/BPMN_removed_short_rows.csv"

# vedo l'encoding del file
input_file_encoding = get_file_encoding(file_path)
print(f"L'encoding del file '{file_path}' è: {input_file_encoding}")

# Save the data of the .csv file in a Variable
df = pd.read_csv(file_path, sep=';', engine='python', encoding=input_file_encoding)

# Prendo solo le utlime due colonne
df = df.iloc[:, [0,-2]]

# Elimino i duplicati
df = df.drop_duplicates()

# Pulisco le colonne dai "^^^" se x è una stringa tolto ^^^ altrimenti ritorno x
df['Labels'] = df['Labels'].fillna('').apply(lambda x: 
                                             x.split("^^^") if isinstance(x, str) else x)

# Mi salvo dominio-etichette solo per controllare i dati
df.to_csv('./Discarded_Datas/BPMN_domains_and_labels',index=False, sep=';')

# Salvo le righe con meno di 4 parole oppure con solo lettere singole o doppie in una struttura
removed_rows_df = df[df['Labels'].apply(lambda x: 
                                        sum(1 for item in x if isinstance(item, str) and len(item) <= 2) > len(x) / 2 or len(x) < 4)]

# Espressione regolare per identificare i valori che iniziano con 's' seguiti da numeri
pattern = r'^s\d+$'

# Filtrare le righe che corrispondono al pattern
df = df[~df['Labels'].apply(lambda x: any(re.match(pattern, item) for item in x if isinstance(item, str)))]

# Salvo le righe eliminate in un .csv
removed_rows_df.to_csv(removed_rows_file_path, index=False, sep=';')

# Mi porto nel dataframe  
df = df[~df['Labels'].apply(lambda x: 
                            sum(1 for item in x if isinstance(item, str) and len(item) <= 2) > len(x) / 2 or len(x) < 4)]

# Tolgo eCH
df = df.drop(df[df['CollectionName']== 'eCH'].index)

# Salvo il file finito in un .csv
df.to_csv(output_file_path, index=False, sep=';')

df.info()
df.head(15)

# Stampo le righe eliminate
print("Righe rimosse:")
print(removed_rows_df)

#print(type(df))
#print(type(df['Labels']))
#print(type(df['Labels'].iloc[2]))
#print(type(df['Labels'].iloc[2][2]))

ModuleNotFoundError: No module named 'pandas'

In [11]:
import ast
import chardet
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

%run "./support_functions.ipynb"

input_file_path = './Output_Files/BPMN_cleaned.csv'

# Creazione del dataset con i suoi metodi richiesti di default
class CustomDataset(Dataset):
    def __init__(self, path):
        
         # Prima di tutto controlliamo che il path esista
        if not os.path.isfile(path):
            # Se il path non è valido allora solleviamo un'eccezione
            raise ValueError('Invalid `path` variable! Needs to be a file')

        input_file_encoding = get_file_encoding(input_file_path)

        # Carico il file .csv
        self.df = pd.read_csv(path, sep=';', engine='python', encoding=get_file_encoding(path))

        self.data = self.df['Labels']
        self.length = len(self.data)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample = self.data[index]
        # Potrei anche usare un tokenizer quì e ritornarlo sotto
        return sample
        
# Creazione del dataloader che sarà necessario ai fini di usare un algoritmo per la divisione di frasi condensate
def create_dataloader(dataframe, batch_size, shuffle=True):
    dataset = CustomDataset(dataframe)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [20]:
import os
import re
import ast
import wordninja
import chardet
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset


##Script che separa parola per parola


%run "./support_functions.ipynb"

input_file_path = './Output_Files/BPMN_cleaned.csv'
output_file_path = './Output_Files/BPMN_uncondensed_sentences_rows.csv'

batch_size = 32

tqdm.pandas()

# Vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)

# Leggo il file CSV in un DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Creazione del dataloader che creerà un dataset utile da passare al 'T5' 
dataloader = create_dataloader(input_file_path, batch_size)
tokenized_data = []

# Rimozione della punteggiatura prima della tokenizzazione
def remove_punctuation(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text)
    else:
        return text 

# Itero ogni batch del dataloader e applichiamo wordninja per separare le frasi condensate
for batch in tqdm(dataloader, desc='Tokenization Progress'):
    cleaned_batch = [remove_punctuation(label) for label in batch]
    tokenized_batch = [wordninja.split(label) for label in cleaned_batch]
    tokenized_data.extend(tokenized_batch)


df['Labels'] = tokenized_data

# Salvo il DataFrame risultante in un nuovo file CSV
df.to_csv(output_file_path, index=False, sep=';')

#print(type(tokenized_data)) #List
#print(type(tokenized_data[2])) #List
#print(type(tokenized_data[2][2])) #String


L'encoding del file './Output_Files/BPMN_cleaned.csv' è: utf-8.
L'encoding del file './Output_Files/BPMN_cleaned.csv' è: utf-8.


Tokenization Progress: 100%|████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:24<00:00, 16.36it/s]


ValueError: Length of values (4716) does not match length of index (12556)

In [21]:
import os
import re
import ast
import wordninja
import chardet
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

##Script che mi ritorna una frase unica

%run "./support_functions.ipynb"

input_file_path = './Output_Files/BPMN_cleaned.csv'
output_file_path = './Output_Files/BPMN_uncondensed2_sentences_rows.csv'

batch_size = 32

tqdm.pandas()

# Vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}.")

# Leggo il file CSV in un DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Rimozione della punteggiatura prima della tokenizzazione
def remove_punctuation(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text)
    else:
        return text 

# Funzione per separare le parole unite dai token
def split_words(text):
    if isinstance(text, str):
        return ' '.join(wordninja.split(text))
    else:
        return text 

# Applica la rimozione della punteggiatura e la separazione delle parole ai token nel DataFrame
df['Labels'] = df['Labels'].progress_apply(remove_punctuation)
df['Labels'] = df['Labels'].progress_apply(split_words)

# Salvo il DataFrame risultante in un nuovo file CSV
df.to_csv(output_file_path, index=False, sep=';')



L'encoding del file './Output_Files/BPMN_cleaned.csv' è: utf-8.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 12556/12556 [01:01<00:00, 205.56it/s]


In [16]:
import os
import re
import ast
import torch
import chardet
import torch.nn as nn
import pandas as pd
from ast import literal_eval
from langdetect import detect
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

##LANGDETECT

%run "./support_functions.ipynb"

input_file_path = './Output_Files/BPMN_uncondensed_sentences_rows.csv'
output_file_path = './Output_Files/BPMN_cleaned_languages.csv'
removed_rows_lang_file_path = "./Discarded_Datas/BPMN_removed_lang_rows.csv"

# Vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}.")

# Leggo il file CSV in un DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Imposto la lingua che vogliamo mantenere e ritorno se la stringa appartiene 
def is_desired_language(text):
    try:
        return detect(text)=='en'
    except:
        return False
    
# Metodo che cicla ogni etichetta
def is_desired_language_list(labels):
    return any(is_desired_language(str(label)) for label in labels)

def filter_dataframe(df):
    # Wrapping the DataFrame filtering process with tqdm
    with tqdm(total=len(df), desc='Filtering DataFrame', unit='row') as pbar:
        df['Labels'] = df['Labels'].apply(ast.literal_eval)
        df['is_desired_language'] = df['Labels'].apply(is_desired_language_list)
        
        discarded_rows = df[~df['is_desired_language']]
        filtered_df = df[df['is_desired_language']].drop(['is_desired_language'], axis=1)

        # Mi salvo le righe eliminate perchè non consone ad 'en'
        discarded_rows.to_csv(removed_rows_lang_file_path, index=False, sep=';')

        # stampo le righe eliminate
        print(f"Sono state eliminate {len(discarded_rows)} righe.")
        print(f"Righe eliminate: \n {discarded_rows}")
        # Update progress bar
        pbar.update(len(df) - len(discarded_rows))

    return filtered_df

# Applico il filtro al DataFrame
filtered_df = filter_dataframe(df)

# Salvo il DataFrame risultante in un nuovo file CSV
filtered_df.to_csv(output_file_path, index=False, sep=';')

L'encoding del file './Output_Files/BPMN_uncondensed_sentences_rows.csv' è: ascii.


Filtering DataFrame:  92%|█████████████████████████████████████████████████████████████████████████      | 11604/12556 [1:00:03<04:55,  3.22row/s]

Sono state eliminate 952 righe.
Righe eliminate: 
       CollectionName                                             Labels  \
17             BPMAI  [Act, i, vida, d, 1, Act, i, vida, d, 2, Act, ...   
27             BPMAI  [An, r, ufer, fol, g, reich, An, ru, f, nicht,...   
55             BPMAI  [Auto, ma, tisch, Ende, 3, Ende, 1, Ende, 2, E...   
70             BPMAI                                                 []   
74             BPMAI  [Ban, f, gene, hm, i, gen, Be, stellan, for, d...   
...              ...                                                ...   
12472        Camunda  [Computer, backend, Computer, backend, Default...   
12493        Camunda  [Mit, g, lied, be, stehen, d, Mit, g, lied, Ne...   
12519        Camunda  [Ly, mp, fk, not, en, status, N, 0, Lymph, kno...   
12542        Camunda  [Ab, we, sen, he, it, G, le, it, zeit, an, tra...   
12549        Camunda  [Ent, it, Ent, it, KO, Modifier, less, old, es...   

       is_desired_language  
17                 

In [24]:
import os
import pandas as pd
import langid
from tqdm import tqdm

##LANGID

input_file_path = './Output_Files/BPMN_uncondensed2_sentences_rows.csv'
output_file_path = './Output_Files/BPMN_cleaned2_languages.csv'
removed_rows_lang_file_path = "./Discarded_Datas/BPMN_removed2_lang_rows.csv"

# Funzione per controllare la lingua di un testo
def is_desired_language(text):
    try:
        lang, _ = langid.classify(text)
        return lang == 'en'
    except:
        return False

def filter_dataframe(df):
    # Wrapping the DataFrame filtering process with tqdm
    with tqdm(total=len(df), desc='Filtering DataFrame', unit='row') as pbar:
        # Applica la funzione is_desired_language a ogni testo del DataFrame
        df['is_desired_language'] = df['Labels'].apply(is_desired_language)
        
        # Seleziona le righe del DataFrame in cui il testo è in inglese
        filtered_df = df[df['is_desired_language']]
        # Salva le righe scartate in un file separato
        discarded_rows = df[~df['is_desired_language']]
        discarded_rows.to_csv(removed_rows_lang_file_path, index=False, sep=';')

        # Stampa le righe eliminate
        print(f"Sono state eliminate {len(discarded_rows)} righe.")
        print(f"Righe eliminate: \n {discarded_rows}")
        
        # Aggiorna la barra di avanzamento
        pbar.update(len(df) - len(discarded_rows))

    return filtered_df

# Leggi il file CSV in un DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python')

# Applica il filtro al DataFrame
filtered_df = filter_dataframe(df)

# Salva il DataFrame risultante in un nuovo file CSV
filtered_df.to_csv(output_file_path, index=False, sep=';')


Filtering DataFrame:  39%|███████████████████████████████▊                                                 | 4924/12556 [00:20<00:31, 243.21row/s]

Sono state eliminate 7632 righe.
Righe eliminate: 
       CollectionName                                             Labels  \
0              BPMAI  A rz t unter such t nach Vol l st n dig ke it ...   
3              BPMAI  A NO A NO Default Input Set Default Input Set ...   
4              BPMAI  Date n be re its vor hand en J a J a J a Kon t...   
8              BPMAI  Analysed ela demand e Cr ation du com pte Est ...   
9              BPMAI  Kun de Kun de Kun de Kun de Kun de Log is tik ...   
...              ...                                                ...   
12551        Camunda  Ange bota us w hlen Ange botS 1 e in hole n An...   
12552        Camunda  j a ne in Regress mg lich Regress pr fen Regre...   
12553        Camunda  3 Ange bot ee in hole n Klein men ge Lager arb...   
12554        Camunda  Bon it t be iSch ufa an fragen Er geb nisan Ba...   
12555        Camunda  ander Rei he An ges tell ter An ges tell ter B...   

       is_desired_language  
0                 

In [25]:
import ast
import chardet
import pandas as pd

#SPLIT IN TRAIN AND TEST

%run "./support_functions.ipynb"

input_file_path = './Output_Files/BPMN_cleaned2_languages.csv'
output_training_file = './Training_Testing_Files/BPMN_training2.csv'
output_testing_file = './Training_Testing_Files/BPMN_testing2.csv'

# vedo l'encoding del file
input_file_encoding = get_file_encoding(input_file_path)
print(f"L'encoding del file '{input_file_path}' è: {input_file_encoding}")

# Loading the DataFrame
df = pd.read_csv(input_file_path, sep=';', engine='python', encoding=input_file_encoding)

# Funzione che divide il DataFrame in proporzion
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# Funzione che da un dataframe ritorna un dataframe di dataframe divisi
def split_by_collectionname(data):
    divided_data = {}
    for collectionname in data['CollectionName'].unique():
        divided_data[collectionname] = data[data['CollectionName'] == collectionname].copy()
    return divided_data

# Funzione che per ogni chiave (collectionname) divide in train e testing
def creating_train_test_dataframe(data, test_ratio):
    training_df = []
    testing_df = []
    # Dividi il DataFrame per CollectionName
    divided_data = split_by_collectionname(data)
    # Per ogni CollectionName, dividi il DataFrame in train e test
    for collectionname, data in divided_data.items():
        train_df, test_df = split_train_test(data, test_ratio)
        training_df.append(train_df)
        testing_df.append(test_df)
        
        training_data=pd.concat(training_df)
        testing_data=pd.concat(testing_df) 
    
    return training_data, testing_data  
        
# Prende randomicamente e in percentuale le righe del df per training e testing
training_data, testing_data = creating_train_test_dataframe(df, 0.27)


# Stampo le prime 20 righe di training e testing
# Stampa il numero dei diversi domini nel DataFrame di testing
print("Numero di diversi domini nel testing_data:")
print(testing_data['CollectionName'].value_counts())
print(testing_data.head())

# Stampa il numero dei diversi domini nel DataFrame di training
print("\nNumero di diversi domini nel training_data:")
print(training_data['CollectionName'].value_counts())
print(training_data.head())

# salva in file rispettivi per training e testing
training_data.to_csv(output_training_file, index=False, sep=';')
testing_data.to_csv(output_testing_file, index=False, sep=';')

L'encoding del file './Output_Files/BPMN_cleaned2_languages.csv' è: ascii
Numero di diversi domini nel testing_data:
CollectionName
BPMAI      1225
Camunda     103
Name: count, dtype: int64
     CollectionName                                             Labels  \
361           BPMAI  810 min Add pasta Boil water with little salt ...   
1841          BPMAI  Check internet connect tion Client Client Conn...   
402           BPMAI                        Pool 1 Pool 1 Pool 2 Pool 2   
4066          BPMAI  Customer WS Customer WS Customer WS Customer W...   
3012          BPMAI  200 am Backup database Shrink the database bac...   

      is_desired_language  
361                  True  
1841                 True  
402                  True  
4066                 True  
3012                 True  

Numero di diversi domini nel training_data:
CollectionName
BPMAI      3315
Camunda     281
Name: count, dtype: int64
     CollectionName                                             Labels  \
1220 